# `transformers` meets `bitsandbytes` for democratzing Large Language Models (LLMs) through 4bit quantization

<center>
<img src="https://github.com/huggingface/blog/blob/main/assets/96_hf_bitsandbytes_integration/Thumbnail_blue.png?raw=true" alt="drawing" width="700" class="center"/>
</center>

Welcome to this notebook that goes through the recent `bitsandbytes` integration that includes the work from XXX that introduces no performance degradation 4bit quantization techniques, for democratizing LLMs inference and training.

In this notebook, we will learn together how to load a large model in 4bit (`gpt-neo-x-20b`) and train it using Google Colab and PEFT library from Hugging Face 🤗.

[In the general usage notebook](https://colab.research.google.com/drive/1ge2F1QSK8Q7h0hn3YKuBCOAS0bK8E0wf?usp=sharing), you can learn how to propely load a model in 4bit with all its variants.

If you liked the previous work for integrating [*LLM.int8*](https://arxiv.org/abs/2208.07339), you can have a look at the [introduction blogpost](https://huggingface.co/blog/hf-bitsandbytes-integration) to lean more about that quantization method.


In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 13.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 80.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver d

First let's load the model we are going to use - GPT-neo-x-20B! Note that the model itself is around 40GB in half precision

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForSequenceClassification

model_id = "huawei-noah/TinyBERT_General_4L_312D"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["query", "key", "value"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 120434 || all params: 12146284 || trainable%: 0.9915295904492271


Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.1 MB/s eta 0:00:00


In [ ]:
import evaluate
import numpy as np
import transformers

metric = evaluate.load("accuracy")

# 4. Define compute_metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)  # Convert logits to predicted class
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# 4. Load GLUE dataset (MRPC task)
from datasets import load_dataset  # Import the load_dataset function

task = "mrpc"
dataset = load_dataset("glue", task)
metric = evaluate.load("glue", task)

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'accuracy',  # Track accuracy as the metric for saving the best model
        'goal': 'maximize'
    },
    'parameters': {
        'optimizer': {
            'values': ['adamw_torch', 'adafactor', 'adamw_hf', 'adamw_8bit', 'sgd']
        },
        'learning_rate': {
            'distribution': 'log_uniform_values',
            'min': 1e-5,
            'max': 1e-3
        },
        'lr_scheduler': {
            'values': [
                'linear',
                'cosine',
                'constant',
                'constant_with_warmup',
                'polynomial'
            ]
        },
        'weight_decay': {
            'values': [0.0, 0.01, 0.001, 0.1]
        },
        'warmup_ratio': {
            'values': [0.05, 0.1, 0.15, 0.2]
        },
        'train_batch_size': {
            'values': [8, 16, 32, 64, 128]
        },
        'gradient_accumulation_steps': {
            'values': [2, 4, 8, 32, 64]
        },
        'lora_r': {
            'values': [8, 16, 32, 64, 128]
        },
        'lora_alpha': {
            'values': [4, 8, 16, 32, 64]
        },
        'lora_dropout': {
            'values': [0.05, 0.1, 0.2]
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="TinyBert 101")

Create sweep with ID: pbit4fos
Sweep URL: https://wandb.ai/garima440-new-york-university/TinyBert%20101/sweeps/pbit4fos


Run the cell below to run the training! For the sake of the demo, we just ran it for few steps just to showcase how to use this integration with existing tools on the HF ecosystem.

In [ ]:
# Set a padding token for the tokenizer
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add a padding token

# 3. Preprocessing function for tokenization
def preprocess_function(examples):
    return tokenizer(
        examples["sentence1"],
        examples["sentence2"],
        truncation=True,
        padding="max_length",  # Ensure uniform input size
        max_length=512,       # Typical BERT max length
    )

# 4. Tokenize dataset
encoded_dataset = dataset.map(preprocess_function, batched=True)

# 5. Data collator
data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        # 6. Define Trainer with TrainingArguments
        trainer = transformers.Trainer(
            model=model,
            train_dataset=encoded_dataset["train"],
            eval_dataset=encoded_dataset["validation"],
            tokenizer=tokenizer,
            data_collator=data_collator,
            args=transformers.TrainingArguments(
                per_device_train_batch_size=config.train_batch_size,  # BERT can handle larger batch sizes
                gradient_accumulation_steps=config.gradient_accumulation_steps,  # Adjust if GPU memory is limited
                warmup_ratio=config.warmup_ratio,
                max_steps=300,
                learning_rate=config.learning_rate,
                fp16=True,  # Enable mixed-precision if supported by your hardware
                logging_steps=50,
                evaluation_strategy="steps",  # Evaluate periodically
                output_dir="./outputs",
                save_steps=100,
                save_total_limit=2,  # Keep only the latest 2 checkpoints
                optim=config.optimizer,
                weight_decay=config.weight_decay,
                lr_scheduler_type=config.lr_scheduler,
            ),
                compute_metrics=compute_metrics,

        )

        # 7. Disable caching for training
        model.config.use_cache = False

        # 8. Train the model
        trainer.train()


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

In [ ]:
wandb.agent(sweep_id, train, count=1)

wandb: Agent Starting Run: m65nty3u with config:
wandb: 	gradient_accumulation_steps: 2
wandb: 	learning_rate: 0.0003916496770060197
wandb: 	lora_alpha: 8
wandb: 	lora_dropout: 0.2
wandb: 	lora_r: 16
wandb: 	lr_scheduler: polynomial
wandb: 	optimizer: adafactor
wandb: 	train_batch_size: 128
wandb: 	warmup_ratio: 0.2
wandb: 	weight_decay: 0


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-15-3801201518a6>:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: WARNING Config item 'gradient_accumulation_steps' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_ratio' was locked by 'sweep' (ignored update).
/usr/loc

Step,Training Loss,Validation Loss,Accuracy,F1
50,0.661800,0.652600,0.683824,0.812227
100,0.602400,0.572950,0.681373,0.810496
150,0.532400,0.499376,0.789216,0.857616
200,0.482300,0.458111,0.813725,0.875410
250,0.459700,0.434768,0.821078,0.877311
300,0.439700,0.430333,0.823529,0.879195


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

eval/accuracy,▁▁▆███
eval/f1,▁▁▆███
eval/loss,█▅▃▂▁▁
eval/runtime,▂▂█▆▂▁
eval/samples_per_second,▇▇▁▃▇█
eval/steps_per_second,▇▇▁▃▇█
train/epoch,▁▁▂▂▄▄▅▅▇▇███
train/global_step,▁▁▂▂▄▄▅▅▇▇███
train/grad_norm,▃▁▁█▃▆
train/learning_rate,██▆▅▃▁
train/loss,█▆▄▂▂▁


In [ ]:
!pip install xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 64.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import time

# Load tokenizer and trained model
tokenizer = AutoTokenizer.from_pretrained("./outputs/checkpoint-300")  # Path to your saved model
model = AutoModelForSequenceClassification.from_pretrained("./outputs/checkpoint-300")
model.eval()  # Set to evaluation mode
model.config.use_cache = True

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers.models.bert.modeling_bert import BertSelfAttention

# # Patch BERT with FlashAttention if supported
# def enable_flash_attention(self, hidden_states, attention_mask=None):
#     from xformers.ops import memory_efficient_attention

#     attention_output = memory_efficient_attention(
#         hidden_states,
#         hidden_states,
#         hidden_states,
#         attn_bias=attention_mask,
#     )
#     return attention_output

# BertSelfAttention.forward = enable_flash_attention

In [ ]:
from xformers.ops import memory_efficient_attention

class FlashSelfAttention(nn.Module):
    def __init__(self, config, device):
        super().__init__()
        if config.hidden_size % config.num_attention_heads != 0:
            raise ValueError(
                f"The hidden size ({config.hidden_size}) is not a multiple of the number of attention heads ({config.num_attention_heads})"
            )
        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = config.hidden_size // config.num_attention_heads
        self.query = nn.Linear(config.hidden_size, config.hidden_size)
        self.key = nn.Linear(config.hidden_size, config.hidden_size)
        self.value = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)

    def forward(self, hidden_states, attention_mask=None, head_mask=None, encoder_hidden_states=None, encoder_attention_mask=None, past_key_value=None, output_attentions=False):  # Add extra arguments
        # Project query, key, value
        q = self.query(hidden_states)
        k = self.key(hidden_states)
        v = self.value(hidden_states)

        # Reshape for multi-head attention
        batch_size, seq_length, hidden_dim = hidden_states.size()
        q = q.view(batch_size, seq_length, self.num_attention_heads, self.attention_head_size).transpose(1, 2)
        k = k.view(batch_size, seq_length, self.num_attention_heads, self.attention_head_size).transpose(1, 2)
        v = v.view(batch_size, seq_length, self.num_attention_heads, self.attention_head_size).transpose(1, 2)

        # FlashAttention computation
        if attention_mask is not None:
            attention_mask = attention_mask[:, None, :, None]  # [batch, head, seq, 1]

        attention_output = memory_efficient_attention(q, k, v, attn_bias=attention_mask)
        attention_output = attention_output.transpose(1, 2).contiguous().view(batch_size, seq_length, hidden_dim)

        # Reshape back to original dimensions
        attention_output = attention_output.transpose(1, 2).contiguous().view(batch_size, seq_length, hidden_dim)
        return (attention_output, )  # You might need to adjust the return value based on the original forward method

In [ ]:

# Function to replace the attention layers with FlashSelfAttention
from torch.nn import Module
import torch.nn as nn

# Function to replace the attention layers with FlashSelfAttention
def replace_attention_layers_with_flash(model, device):
    """
    Replace the attention layers in the model with FlashSelfAttention.

    Args:
        model (nn.Module): The model to modify.

    Returns:
        model (nn.Module): The model with replaced attention layers.
    """
    for name, module in model.named_modules():
        # Check if the module has the self-attention component
        if isinstance(module, nn.Module) and hasattr(module, 'attention') and hasattr(module.attention, 'self'):
            print(f"Replacing attention layer in module: {name}")
            # Replace the default self-attention with FlashSelfAttention
            # Use a lambda function to adapt the forward call
            # Pass the device argument to FlashSelfAttention
            # Create a FlashSelfAttention instance and move it to the device
            flash_attention = FlashSelfAttention(model.config, device).to(device)

            # Assign the FlashSelfAttention instance to the module
            module.attention.self = flash_attention

    return model

In [ ]:
model = replace_attention_layers_with_flash(model, device)

Replacing attention layer in module: bert.encoder.layer.0
Replacing attention layer in module: bert.encoder.layer.1
Replacing attention layer in module: bert.encoder.layer.2
Replacing attention layer in module: bert.encoder.layer.3


In [ ]:
import time
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)





def measure_latency(model, tokenizer, sentence1, sentence2, iterations=100):
    # Tokenize input
    inputs = tokenizer(sentence1, sentence2, return_tensors="pt", truncation=True, padding=True, max_length=512)

    inputs = inputs.to(device)

    # inputs = inputs.type(torch.float16)  # or torch.bfloat16
    # Convert only the relevant tensors to float16
    # Cast only specific tensors to float16
    for key in ['attention_mask']:  # Only cast attention_mask if needed
        if key in inputs and isinstance(inputs[key], torch.Tensor):
            inputs[key] = inputs[key].type(torch.float16)

    # Warm-up runs
    for _ in range(10):
        with torch.no_grad():
            results = model(**inputs)

    # Measure latency
    torch.cuda.synchronize()
    start_time = time.perf_counter()
    for _ in range(iterations):
        with torch.no_grad():
            results = model(**inputs)
    torch.cuda.synchronize()
    end_time = time.perf_counter()

    avg_latency = (end_time - start_time) / iterations
    print(f"Average Inference Latency: {avg_latency:.6f} seconds")

# Prepare example input (take a sample from the dataset)
sentence1 = dataset["test"][13]["sentence1"]
sentence2 = dataset["test"][13]["sentence2"]

measure_latency(model, tokenizer, sentence1, sentence2)

Average Inference Latency: 0.003916 seconds
